Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [24]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [41]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [42]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  # One Hot encoding
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [22]:
print(train_dataset[0])

[-0.5        -0.49607843 -0.5        -0.42156863  0.31568629  0.46470588
 -0.30784315 -0.17058824  0.5         0.19803922 -0.46470588  0.20980392
  0.5        -0.23333333 -0.22941177  0.5         0.20980392 -0.46078432
  0.20980392  0.5        -0.21372549 -0.43333334  0.37058824  0.40980393
 -0.33529413 -0.5        -0.49215686 -0.5        -0.49607843 -0.47647059
 -0.5        -0.19803922  0.5         0.5         0.01764706  0.16666667
  0.5         0.44901961 -0.13921569  0.45294118  0.5         0.09607843
  0.09607843  0.5         0.45294118 -0.13529412  0.45294118  0.5
  0.08431373 -0.19019608  0.5         0.5        -0.04901961 -0.5
 -0.47254902 -0.49607843 -0.49607843 -0.49607843 -0.5        -0.43333334
  0.19019608  0.31960785 -0.3392157  -0.22156863  0.36666667  0.09215686
 -0.48039216  0.12352941  0.40196079 -0.2764706  -0.2764706   0.40196079
  0.12352941 -0.47647059  0.10392157  0.35490197 -0.25686276 -0.44509804
  0.2372549   0.27254903 -0.35882354 -0.5        -0.49607843 -0.4

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [17]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


ValueError: Fetch argument <tensorflow.python.framework.ops.Operation object at 0x143343ed0> of <tensorflow.python.framework.ops.Operation object at 0x143343ed0> cannot be interpreted as a Tensor. (Operation name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_2/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_3/ApplyGradientDescent"
 is not an element of this graph.)

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.649601
Minibatch accuracy: 7.8%
Validation accuracy: 6.5%
Minibatch loss at step 500: 1.159378
Minibatch accuracy: 78.9%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 1.508133
Minibatch accuracy: 76.6%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.611867
Minibatch accuracy: 83.6%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 0.837573
Minibatch accuracy: 82.0%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 1.154386
Minibatch accuracy: 73.4%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 0.881322
Minibatch accuracy: 79.7%
Validation accuracy: 78.8%
Test accuracy: 86.5%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [12]:
batch_size = 128
num_hidden = 1024

graph = tf.Graph()
with graph.as_default():
  #Inputs
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
  layer1_biases = tf.Variable(tf.zeros([num_hidden]))
  layer2_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  layer2_biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  def fp(data):
    a1 = tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    return tf.matmul(a1, layer2_weights) + layer2_biases
  a2 = fp(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(a2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(a2)
  valid_prediction = tf.nn.softmax(fp(tf_valid_dataset))
  test_prediction = tf.nn.softmax(fp(tf_test_dataset))

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 333.092346
Minibatch accuracy: 10.2%
Validation accuracy: 32.0%
Minibatch loss at step 500: 16.306318
Minibatch accuracy: 82.8%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 12.893785
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 6.581551
Minibatch accuracy: 88.3%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 3.763221
Minibatch accuracy: 89.8%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 7.757624
Minibatch accuracy: 80.5%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 2.825783
Minibatch accuracy: 83.6%
Validation accuracy: 82.5%
Test accuracy: 89.8%


In [43]:
# normalizing to [0,1]
train_dataset = (train_dataset +1)/2
test_dataset = (test_dataset +1)/2
valid_dataset = (valid_dataset +1)/2
print(test_dataset[0])

[ 0.25        0.25        0.25        0.25        0.25        0.25588235
  0.25        0.55196077  0.75        0.74215686  0.75        0.75        0.75
  0.75        0.75        0.75        0.75        0.75        0.75
  0.73235297  0.6754902   0.58137256  0.44607842  0.27941176  0.25
  0.25588235  0.25        0.25        0.25        0.25        0.25        0.25
  0.25        0.25        0.25        0.47941175  0.55392158  0.5892157
  0.75        0.74803925  0.75        0.59705883  0.54215688  0.55196077
  0.55000001  0.56568629  0.60294116  0.68333334  0.75        0.74803925
  0.75        0.68529415  0.42647058  0.25        0.25588235  0.25        0.25
  0.25        0.25        0.25        0.25        0.25        0.25
  0.25588235  0.25        0.43039215  0.75        0.75        0.72058821
  0.29117647  0.25        0.25196078  0.25        0.25        0.25
  0.26372549  0.36764705  0.61078429  0.75        0.74607843  0.75
  0.5147059   0.25        0.25392157  0.25        0.25        0.

In [47]:
# auto encoder 
batch_size = 128
num_hidden = 32

def encoding_accuracy(pred, labels):
    return 100.0 * sum([np.array_equal(pred[i],labels[i]) for i in xrange(len(labels))]) / len(labels)
# print(encoding_accuracy(test_dataset, test_dataset))

def x_entropy(output, target):
  with tf.name_scope("xentropy_loss"):
    net_output_tf = tf.convert_to_tensor(output, name='input')
    target_tf = tf.convert_to_tensor(target, name='target')
    cross_entropy = tf.add(tf.mul(tf.log(net_output_tf, name='log_output'),
                                  target_tf),
                           tf.mul(tf.log(1 - net_output_tf),
                                  (1 - target_tf)))
    return -1 * tf.reduce_mean(tf.reduce_sum(cross_entropy, 1),
                                 name='xentropy_mean')

graph = tf.Graph()
with graph.as_default():
  #Inputs
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
  layer1_biases = tf.Variable(tf.zeros([num_hidden]))
  layer2_weights = tf.Variable(tf.truncated_normal([num_hidden, image_size * image_size]))
  layer2_biases = tf.Variable(tf.zeros([image_size * image_size]))

  classifier_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  classifier_biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  def autoencode(data):
    a1 = tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    return tf.matmul(a1, layer2_weights) + layer2_biases

  a2 = autoencode(tf_train_dataset)
  autoencoder_loss = x_entropy(a2, tf_train_dataset)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(autoencoder_loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(a2)
  valid_prediction = tf.nn.softmax(autoencode(tf_valid_dataset))
  test_prediction = tf.nn.softmax(autoencode(tf_test_dataset))

In [49]:
print(x_entropy(train_dataset[0],train_dataset[0]))

ValueError: Invalid reduction dimension 1 for input with 1 dimensions

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, autoencoder_loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % encoding_accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % encoding_accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: nan
Minibatch accuracy: 0.0%
Validation accuracy: 0.0%
Minibatch loss at step 500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 0.0%


KeyboardInterrupt: 